# **Initialization**

In [ ]:
!pip install langchain huggingface_hub sentence-transformers transformers langchain-community faiss-cpu faiss-gpu langchain-huggingface torch
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

# **RAG + LLM for payload generation**
## Text Splitter

In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFaceEndpoint
from langchain.vectorstores import FAISS
from langchain.text_splitter import MarkdownTextSplitter
from langchain.schema import Document
from IPython.display import Markdown, display
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import gc


# Funzione per caricare i file .md dalla directory manualmente
def load_markdown_documents(directory: str):
    documents = []
    for filename in os.listdir(directory):
        if filename.endswith(".md"):
            filepath = os.path.join(directory, filename)
            with open(filepath, "r", encoding="utf-8") as file:
                content = file.read()
                documents.append(Document(page_content=content, metadata={"source": filename}))
    return documents

def create_retriever(documents):
    print("Inizio creazione del retriever...")

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    splitter = MarkdownTextSplitter(chunk_size=450, chunk_overlap=80)
    split_docs = splitter.split_documents(documents)
    print(f"Numero di documenti : {len(split_docs)}")

    texts = [doc.page_content for doc in split_docs]
    vector_store = FAISS.from_texts(texts, embeddings)
    #restituisce i top 3
    retriever = vector_store.as_retriever(search_kwargs={"k": 3})  


    #from langchain.vectorstores import Annoy

    #vector_store = Annoy.from_texts(texts, embeddings)
    #retriever = vector_store.as_retriever()

    print("Retriever creato con successo")
    return retriever


#Caricamento del modello generativo
def create_huggingface_model_local():
    global model
    #model_name = "EleutherAI/gpt-neo-1.3B"
    #model_name = "KimByeongSu/gpt-neo-1.3B_LAMA_TREx_finetuning_MAGNET_same"
    #model_name = "KimByeongSu/gpt-neo-2.7B_LAMA_TREx_finetuning_MAGNET"
    model_name ="ricepaper/vi-gemma-2b-RAG"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    def generate_text(prompt, max_length_i=1350, temperature_i=0.95):
        print("Avvio del modello per la generazione del testo...")
        inputs = tokenizer(prompt, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model.generate(
            inputs['input_ids'],
            attention_mask=inputs['attention_mask'],  # Passa l'attenzione mask
            max_length=max_length_i,            # Ridotto ulteriormente per evitare output lunghi e ripetitivi
            do_sample=True,            # Campionamento abilitato per la varietà
            pad_token_id=tokenizer.eos_token_id,
            temperature=temperature_i,           # Migliora la creatività
            top_k=150,                  # Aumentato per fornire più opzioni durante la generazione
            top_p=0.95,                 # Probabilità cumulativa controllata
            repetition_penalty=2.6,     # Penalizza ripetizioni eccessive di token
            num_beams=5,                # Beam search per maggiore coerenza
            early_stopping=True        # Ferma la generazione quando una condizione è soddisfatta
            )
            return tokenizer.decode(outputs[0], skip_special_tokens=True)
    return generate_text

def create_rag_chain(retriever, generate_text_fn):
    def rag_chain(query):
        relevant_docs = retriever.invoke(query)

        context = "\n".join([doc.page_content for doc in relevant_docs])
        full_prompt = f"Context:{context}\n\n Question: {query} \n\n Your response: "
        
        generated_text = generate_text_fn(full_prompt)
        return generated_text, relevant_docs
    
    return rag_chain

# Funzione per fare una query al sistema RAG
def query_rag(chain, query):
    result, source_documents = chain(query)
    
    return result, source_documents

# Funzione per svuotare la RAM della GPU
def optimizer_gpu():
        model.to("cpu")
        print("Memoria dopo la generazione")
        print(f"Memory allocated: {torch.cuda.memory_allocated()}")
        print(f"Memory reserved: {torch.cuda.memory_reserved()}")
        torch.cuda.empty_cache()
        gc.collect()
        print("Memoria dopo l'ottimizzazione")
        print(f"Memory allocated: {torch.cuda.memory_allocated()}")
        print(f"Memory reserved: {torch.cuda.memory_reserved()}")
        model.to("cuda")
        

# Esecuzione del sistema
while True:
    if __name__ == "__main__":
        #IMPORTAZIONE DEL DATASET
        #directory = "/kaggle/input/docs-tools"
        directory="/kaggle/input/docs-tools-filtrato-2"

        documents = load_markdown_documents(directory)
        retriever = create_retriever(documents)

        # Usa il modello locale invece dell'endpoint remoto
        generate_text_fn = create_huggingface_model_local()

        rag_chain = create_rag_chain(retriever, generate_text_fn)

        query = input("Inserisci una query :")
        response, docs = query_rag(rag_chain, query)

        print("Risposta generata: \n", response)

        print(f"Numero di documenti utilizzati: {len(docs)}")
        j=0

        for doc in docs:
            j=j+1
            print(f"\nDocumento sorgente {j}:", doc.page_content)
               

            
        
        #Visualizza in markdown
        i=int(input("Digita un numero maggiore di 0 per visualizzare la risposta in markdown"))
        if(i>0):
                print("         =================================================")
                display(Markdown(response))
                i=0
                i=int(input("Vuoi salvare l'output come file markdown? In tal caso digita un numero maggiore di 0"))
                if(i>0):
                    # Salvataggio del contenuto nel file Markdown
                    filename=input("Inserisci il nome del file")
                    with open(filename, "w") as file:
                        file.write(response)
        i=0
        optimizer_gpu()